# Part 3.1 !! Pandas DataFrame to PostgreSQL using Python

<h3>Insert Bulk Data Using executemany() Into PostgreSQL Database</h3>

<h3>Prerequisites</h3>

<h4>(a) Python 3.8.3 </h4>

[Anaconda download link](https://www.anaconda.com/products/individual)

<h4>(b) PostgreSQL 13</h4>

[Download link](https://www.postgresql.org/download/)

<b>(c) Psycopg2 : </b> To install <b>Psycopg2</b> use the command: <b>pip install psycopg2</b>

<h3>Insert Bulk Data Using executemany()</h3>

<h3>Step 1 : Prepare or identify your data</h3>

To begin, prepare or identify the CSV file that you'd like to import to PostgreSQL database. For example, we loaded iris data from GitHub.

In [1]:
# import sys to get more detailed Python exception info
import sys
# import the connect library for psycopg2
import psycopg2
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd
irisData = pd.read_csv('https://github.com/Muhd-Shahid/Write-Raw-File-into-Database-Server/raw/main/iris.csv',index_col=False)
irisData.dtypes

sepal_length    float64
sepal_width     float64
petal_length    float64
petal_width     float64
species          object
dtype: object

<h3>Step 2: Specify the connection parameters</h3>

In [2]:
# Note: please change your database, username & password as per your own values

conn_params_dic = {
    "host"      : "localhost",
    "database"  : "irisdb",
    "user"      : "postgres",
    "password"  : "Passw0rd"
}

<h3> Step 3 : Support Functions </h3>

In [6]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    
# Define a connect function for PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successfully..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

# Define function using cursor.executemany() to insert the dataframe
def execute_many(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

<h3> Step 4 : Perform main task</h3>

In [4]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute_many(conn, irisData, 'iris')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successful..................
Data inserted using execute_many() successfully...


<H3>Step 5: Query the database to check our work</H3>

Again, let's query the database to make sure that our inserted data has been saved correctly.

In [20]:
# Connect to the database
conn = connect(conn_params_dic)
cursor = conn.cursor()

# Execute query
sql = "SELECT * FROM iris"
cursor.execute(sql)

# Fetch all the records
tuples = cursor.fetchall()

# list of columns
cols = list(irisData.columns)

irisdf = pd.DataFrame(tuples,columns=cols)
print()
print(irisdf.head())

# Close the cursor
cursor.close()

# Close the conn
conn.close()

Connecting to the PostgreSQL...........
Connection successful..................

  sepal_length sepal_width petal_length petal_width      species
0          5.1         3.5          1.4         0.2  setosa     
1          4.9         3.0          1.4         0.2  setosa     
2          4.7         3.2          1.3         0.2  setosa     
3          4.6         3.1          1.5         0.2  setosa     
4          5.0         3.6          1.4         0.2  setosa     


<b>Conclusion : </b> This ends our Part 3 on Insert Bulk Data using executemany(). In this tutorial we have learned how to insert data into PostgreSQL database using python.

<h4> Next : Part 3.2 - Insert Bulk Data Using execute_batch() method into PostgreSQL Database using Python</h4>

<b>Stay Positive !! Stay Safe !! Keep Learning :):)